In [1]:
import pandas as pd
import datetime
import re
from utils import *

# PARAMETERS 

In [2]:
time = "t9_2019"

# STOCK PROCESSING

In [3]:
stock = get_df_from_mysql(database_name="Stock", table_name=time)
stock = split_type_and_main_account(stock, "AccountCode")
print(stock.dtypes)
stock

TradingDate      object
AccountCode      object
ShareCode        object
ShareBalance    float64
main_account     object
type_account     object
dtype: object


,TradingDate,AccountCode,ShareCode,ShareBalance,main_account,type_account
0,2019-09-30T00:00:00,870374c58baf4321909a5df8551cca411,D2D,500.0,870374c58baf4321909a5df8551cca41,1
1,2019-09-30T00:00:00,e04aac21665645d19714f21f57fd7d626,SDI,600.0,e04aac21665645d19714f21f57fd7d62,6
2,2019-09-30T00:00:00,bf11920b721d4367aea6be3afa0696e46,VRE,300.0,bf11920b721d4367aea6be3afa0696e4,6
3,2019-09-30T00:00:00,227f43173b5c4cc2a4e4c578499d74c51,SJS,14000.0,227f43173b5c4cc2a4e4c578499d74c5,1
4,2019-09-30T00:00:00,b8169237d1204116b9aac27bf34d53996,SCR,10000.0,b8169237d1204116b9aac27bf34d5399,6
...,...,...,...,...,...,...
44199,2019-09-30T00:00:00,f8e9a641746847de9c26079cb948c39a1,DXG,4000.0,f8e9a641746847de9c26079cb948c39a,1
44200,2019-09-30T00:00:00,d685cf0b5e224056b5ed6b9f45ca56146,VGC,170.0,d685cf0b5e224056b5ed6b9f45ca5614,6
44201,2019-09-30T00:00:00,542a779b762e45378db7d4dfbdd2da161,HNM,64800.0,542a779b762e45378db7d4dfbdd2da16,1
44202,2019-09-30T00:00:00,1660ee09a4644889bf52a115098175a41,DQC,95830.0,1660ee09a4644889bf52a115098175a4,1


In [4]:
stock = stock.groupby(['main_account', 'ShareCode'])['ShareBalance'].sum().to_frame('ShareBalance').reset_index()
stock

,main_account,ShareCode,ShareBalance
0,0001898ab3584ff6a99ec11ffcb1f09c,CTG,1.0
1,0001898ab3584ff6a99ec11ffcb1f09c,FLC,401.0
2,0001898ab3584ff6a99ec11ffcb1f09c,PGC,5.0
3,0001898ab3584ff6a99ec11ffcb1f09c,PVI,50.0
4,0001898ab3584ff6a99ec11ffcb1f09c,PVS,2140.0
...,...,...,...
42655,fff283aac89f4f82ab8b931dd9a9e41e,HUT,700.0
42656,fff283aac89f4f82ab8b931dd9a9e41e,PVV,1700.0
42657,fff283aac89f4f82ab8b931dd9a9e41e,VRG,12400.0
42658,fff942970c8d4976993aa8e949f2c228,IDI,7.0


# CASH PROCESSING

In [5]:
cash = get_df_from_mysql(database_name="Cash", table_name=time)
cash = split_type_and_main_account(cash, "AccountCode")
print(cash.dtypes)
cash

TradingDate      object
AccountCode      object
CashBalance     float64
main_account     object
type_account     object
dtype: object


,TradingDate,AccountCode,CashBalance,main_account,type_account
0,2019-09-30T00:00:00,d057a03df81a4d1bbc8c7c2736189a201,6.384183e+11,d057a03df81a4d1bbc8c7c2736189a20,1
1,2019-09-30T00:00:00,64efa1e427fd49d593b4af17d1b6a6fc1,7.347715e+06,64efa1e427fd49d593b4af17d1b6a6fc,1
2,2019-09-30T00:00:00,12a7d31ec56a48068df13719f000b4471,7.102530e+05,12a7d31ec56a48068df13719f000b447,1
3,2019-09-30T00:00:00,1859c8d9f10945379c7d8e10bb3641af1,2.947980e+07,1859c8d9f10945379c7d8e10bb3641af,1
4,2019-09-30T00:00:00,c02fee701cec434fa973e19de661f9451,1.417460e+05,c02fee701cec434fa973e19de661f945,1
...,...,...,...,...,...
32321,2019-09-30T00:00:00,a93cea6987f44f46aeb1cb50c35023b31,1.185411e+06,a93cea6987f44f46aeb1cb50c35023b3,1
32322,2019-09-30T00:00:00,134067f2430a4a108a5770f435cac2b01,4.171200e+04,134067f2430a4a108a5770f435cac2b0,1
32323,2019-09-30T00:00:00,b850c7a823ff435398295dad166e25301,4.386000e+03,b850c7a823ff435398295dad166e2530,1
32324,2019-09-30T00:00:00,235a05c8a4e2415eb0f5a3dbd1d335c21,2.154200e+04,235a05c8a4e2415eb0f5a3dbd1d335c2,1


In [6]:
cash = cash.groupby(['main_account'])['CashBalance'].sum().to_frame('CashBalance').reset_index()
cash

,main_account,CashBalance
0,0001898ab3584ff6a99ec11ffcb1f09c,417980.0
1,00029e5f2da54460a1d7dd1a491d991f,152855.0
2,0002b6649dfc4fdb9f6d284761a74a9b,171760.0
3,00057207ba05405a911387bbcb3861bc,23146.0
4,0006f6007bf640339316572984c76de2,125163.0
...,...,...
26811,fff5749810714e77b4a0822b22d81ce5,849891.0
26812,fff592cb8c4b4a489581113c9c82e022,102815.0
26813,fff942970c8d4976993aa8e949f2c228,576.0
26814,fff95069a1554792bfe041e2f7b184bc,466813.0


# CUSTOMER PROCESSING

In [7]:
customer = get_df_from_mysql(database_name="Customer", table_name=time)
customer

,IdIssueDate,Accounts,Age,CustomerType,IdType,IdIssuePlace,Address,Gender
0,2006-12-25T00:00:00,"['d057a03df81a4d1bbc8c7c2736189a201', 'd057a03...",0,ORGANIZATION,3,Trung tâm lưu ký Chứng khoán Việt Nam,None,None
1,2010-12-06T00:00:00,['64efa1e427fd49d593b4af17d1b6a6fc1'],57,INDIVIDUAL,1,CA Hà Nội,"Số 2, Ngõ 44 Hào Nam, Ô Chợ Dừa, Q.Đống Đa, Hà...",F
2,2005-03-25T00:00:00,"['12a7d31ec56a48068df13719f000b4471', '12a7d31...",54,INDIVIDUAL,1,CA Hà Nội,"Số 18, Ngõ 32 Cát Linh, Đống Đa, Hà Nội",F
3,1991-10-17T00:00:00,['1859c8d9f10945379c7d8e10bb3641af1'],58,INDIVIDUAL,1,CA Hà Nội,"13B Hàng Cân, Q.Hoàn Kiếm, Hà Nội",M
4,2006-01-10T00:00:00,['c02fee701cec434fa973e19de661f9451'],34,INDIVIDUAL,1,CA Hà Nội,"Số 6 Ngõ 127 Hào Nam, Q. Đống Đa, Hà Nội",F
...,...,...,...,...,...,...,...,...
68373,2008-12-01T00:00:00,['d5809ef6bb32488eab1691e7917f8ed21'],24,ORGANIZATION,2,Trung tâm lưu ký CK,"Sewei 2, 8501 SP Joure, The Netherlands (quốc ...",O
68374,2018-06-14T00:00:00,['b5818a855a5746a88bd4309c52e36ac01'],49,INDIVIDUAL,2,Trung tâm Lưu ký CK,"Block 3B 06-05 The Estella, Xa lo Ha Noi, An P...",F
68375,2017-07-10T00:00:00,['70b15e116f404f788df1ff9ad5c34bd81'],0,ORGANIZATION,3,Sở KH & ĐT TP Hà Nội,None,None
68376,2017-07-10T00:00:00,['4389950902f14b26a97f8a51937ec13b1'],0,ORGANIZATION,3,Sở KH & ĐT TP Hà Nội,None,None


# LOAN PROCESSING

In [8]:
loan = get_df_from_mysql(database_name="Loan", table_name=time)
loan = split_type_and_main_account(loan, "AccountCode")
print(loan.dtypes)
loan

TradingDate      object
AccountCode      object
LoanBalance     float64
main_account     object
type_account     object
dtype: object


,TradingDate,AccountCode,LoanBalance,main_account,type_account
0,2019-09-30T00:00:00,f56273a80d224d918c8692e7f5ebbf726,2.421528e+08,f56273a80d224d918c8692e7f5ebbf72,6
1,2019-09-30T00:00:00,ba5a369055924e47824901ea20b6cf716,2.759968e+07,ba5a369055924e47824901ea20b6cf71,6
2,2019-09-30T00:00:00,ef114c87b4fd40d9b6fc74eba47b43d16,5.857814e+07,ef114c87b4fd40d9b6fc74eba47b43d1,6
3,2019-09-30T00:00:00,2a5be18fc9ce4badbbde37df4391eebf6,3.957052e+06,2a5be18fc9ce4badbbde37df4391eebf,6
4,2019-09-30T00:00:00,d9c31175301542db8527f1afbc306cc36,1.418688e+08,d9c31175301542db8527f1afbc306cc3,6
...,...,...,...,...,...
5411,2019-09-30T00:00:00,e8642f8aeef4456084fe257aaf4c502d6,5.923758e+07,e8642f8aeef4456084fe257aaf4c502d,6
5412,2019-09-30T00:00:00,3bc87330b5704b2ea13317048eabb0b96,6.331475e+08,3bc87330b5704b2ea13317048eabb0b9,6
5413,2019-09-30T00:00:00,eb20d0c3be6f4e2f994978ac71fb23036,2.985454e+09,eb20d0c3be6f4e2f994978ac71fb2303,6
5414,2019-09-30T00:00:00,887140035b454595a5e679b89a04b9626,1.953111e+08,887140035b454595a5e679b89a04b962,6


# DATA PROCESSING

In [9]:
data = customer[['Gender', 'Age', 'IdIssuePlace', 'Accounts']]
# Get accounts
def get_account_from_string(account_string):
    account_string = account_string[2:-2]
    account_string = account_string.replace("'","")
    account_string = list(account_string.split(","))
    return account_string
data['Account'] = data['Accounts'].apply(lambda x: get_account_from_string(x))
data

c:\users\natuh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Gender,Age,IdIssuePlace,Accounts,Account
0,None,0,Trung tâm lưu ký Chứng khoán Việt Nam,"['d057a03df81a4d1bbc8c7c2736189a201', 'd057a03...","[d057a03df81a4d1bbc8c7c2736189a201, d057a03df..."
1,F,57,CA Hà Nội,['64efa1e427fd49d593b4af17d1b6a6fc1'],[64efa1e427fd49d593b4af17d1b6a6fc1]
2,F,54,CA Hà Nội,"['12a7d31ec56a48068df13719f000b4471', '12a7d31...","[12a7d31ec56a48068df13719f000b4471, 12a7d31ec..."
3,M,58,CA Hà Nội,['1859c8d9f10945379c7d8e10bb3641af1'],[1859c8d9f10945379c7d8e10bb3641af1]
4,F,34,CA Hà Nội,['c02fee701cec434fa973e19de661f9451'],[c02fee701cec434fa973e19de661f9451]
...,...,...,...,...,...
68373,O,24,Trung tâm lưu ký CK,['d5809ef6bb32488eab1691e7917f8ed21'],[d5809ef6bb32488eab1691e7917f8ed21]
68374,F,49,Trung tâm Lưu ký CK,['b5818a855a5746a88bd4309c52e36ac01'],[b5818a855a5746a88bd4309c52e36ac01]
68375,None,0,Sở KH & ĐT TP Hà Nội,['70b15e116f404f788df1ff9ad5c34bd81'],[70b15e116f404f788df1ff9ad5c34bd81]
68376,None,0,Sở KH & ĐT TP Hà Nội,['4389950902f14b26a97f8a51937ec13b1'],[4389950902f14b26a97f8a51937ec13b1]


In [10]:
# Ungroup account into new rows
data = data.Account.apply(pd.Series).merge(data, right_index = True, left_index = True).drop(["Account"], axis = 1)\
.melt(id_vars = ['Gender', 'IdIssuePlace', 'Age', 'Accounts'], value_name = "Account")\
.drop("variable", axis = 1).dropna(subset=['Account'])
data

,Gender,IdIssuePlace,Age,Accounts,Account
0,None,Trung tâm lưu ký Chứng khoán Việt Nam,0,"['d057a03df81a4d1bbc8c7c2736189a201', 'd057a03...",d057a03df81a4d1bbc8c7c2736189a201
1,F,CA Hà Nội,57,['64efa1e427fd49d593b4af17d1b6a6fc1'],64efa1e427fd49d593b4af17d1b6a6fc1
2,F,CA Hà Nội,54,"['12a7d31ec56a48068df13719f000b4471', '12a7d31...",12a7d31ec56a48068df13719f000b4471
3,M,CA Hà Nội,58,['1859c8d9f10945379c7d8e10bb3641af1'],1859c8d9f10945379c7d8e10bb3641af1
4,F,CA Hà Nội,34,['c02fee701cec434fa973e19de661f9451'],c02fee701cec434fa973e19de661f9451
...,...,...,...,...,...
204177,M,CA TP.HCM,67,"['d30eb237c0e541c5b338b3bdfe75f6ea1', 'd30eb23...",d30eb237c0e541c5b338b3bdfe75f6ea8
204244,M,CA HCM,33,"['537b90d01f534a35a671fef4057e54811', '537b90d...",537b90d01f534a35a671fef4057e54818
204299,M,CA TP HCM,32,"['78f1df56ead44d40922351d947a9976a1', '78f1df5...",78f1df56ead44d40922351d947a9976a8
204565,F,Cục CS ĐKQL CT và DLQG về DC,34,"['f90032b910014018858a92cf2d1976861', 'f90032b...",f90032b910014018858a92cf2d1976868


In [11]:
data = split_type_and_main_account(data, "Account")
data

,Gender,IdIssuePlace,Age,Accounts,Account,main_account,type_account
0,None,Trung tâm lưu ký Chứng khoán Việt Nam,0,"['d057a03df81a4d1bbc8c7c2736189a201', 'd057a03...",d057a03df81a4d1bbc8c7c2736189a201,d057a03df81a4d1bbc8c7c2736189a20,1
1,F,CA Hà Nội,57,['64efa1e427fd49d593b4af17d1b6a6fc1'],64efa1e427fd49d593b4af17d1b6a6fc1,64efa1e427fd49d593b4af17d1b6a6fc,1
2,F,CA Hà Nội,54,"['12a7d31ec56a48068df13719f000b4471', '12a7d31...",12a7d31ec56a48068df13719f000b4471,12a7d31ec56a48068df13719f000b447,1
3,M,CA Hà Nội,58,['1859c8d9f10945379c7d8e10bb3641af1'],1859c8d9f10945379c7d8e10bb3641af1,1859c8d9f10945379c7d8e10bb3641af,1
4,F,CA Hà Nội,34,['c02fee701cec434fa973e19de661f9451'],c02fee701cec434fa973e19de661f9451,c02fee701cec434fa973e19de661f945,1
...,...,...,...,...,...,...,...
204177,M,CA TP.HCM,67,"['d30eb237c0e541c5b338b3bdfe75f6ea1', 'd30eb23...",d30eb237c0e541c5b338b3bdfe75f6ea8,d30eb237c0e541c5b338b3bdfe75f6ea,8
204244,M,CA HCM,33,"['537b90d01f534a35a671fef4057e54811', '537b90d...",537b90d01f534a35a671fef4057e54818,537b90d01f534a35a671fef4057e5481,8
204299,M,CA TP HCM,32,"['78f1df56ead44d40922351d947a9976a1', '78f1df5...",78f1df56ead44d40922351d947a9976a8,78f1df56ead44d40922351d947a9976a,8
204565,F,Cục CS ĐKQL CT và DLQG về DC,34,"['f90032b910014018858a92cf2d1976861', 'f90032b...",f90032b910014018858a92cf2d1976868,f90032b910014018858a92cf2d197686,8


In [12]:
# 
data = data.loc[data['type_account'].isin(["1","6"])]
data = data.drop_duplicates(subset='main_account', keep='last') 
data

,Gender,IdIssuePlace,Age,Accounts,Account,main_account,type_account
0,None,Trung tâm lưu ký Chứng khoán Việt Nam,0,"['d057a03df81a4d1bbc8c7c2736189a201', 'd057a03...",d057a03df81a4d1bbc8c7c2736189a201,d057a03df81a4d1bbc8c7c2736189a20,1
1,F,CA Hà Nội,57,['64efa1e427fd49d593b4af17d1b6a6fc1'],64efa1e427fd49d593b4af17d1b6a6fc1,64efa1e427fd49d593b4af17d1b6a6fc,1
2,F,CA Hà Nội,54,"['12a7d31ec56a48068df13719f000b4471', '12a7d31...",12a7d31ec56a48068df13719f000b4471,12a7d31ec56a48068df13719f000b447,1
3,M,CA Hà Nội,58,['1859c8d9f10945379c7d8e10bb3641af1'],1859c8d9f10945379c7d8e10bb3641af1,1859c8d9f10945379c7d8e10bb3641af,1
4,F,CA Hà Nội,34,['c02fee701cec434fa973e19de661f9451'],c02fee701cec434fa973e19de661f9451,c02fee701cec434fa973e19de661f945,1
...,...,...,...,...,...,...,...
136350,M,CA QUANG NGAI,33,"['f0b9fe2ce784415e90bdd38f83012d421', 'f0b9fe2...",f0b9fe2ce784415e90bdd38f83012d426,f0b9fe2ce784415e90bdd38f83012d42,6
136380,F,CA TP.HCM,31,"['1badc0f1686b4394aa721cd7ccab005e1', '1badc0f...",1badc0f1686b4394aa721cd7ccab005e6,1badc0f1686b4394aa721cd7ccab005e,6
136411,F,TIỀN GIANG,29,"['d4f4fe13800c4569bc471e3dea2210571', 'd4f4fe1...",d4f4fe13800c4569bc471e3dea2210576,d4f4fe13800c4569bc471e3dea221057,6
136477,M,CA HCM,42,"['2271285acdda4e6287e69478dd51d7831', '2271285...",2271285acdda4e6287e69478dd51d7836,2271285acdda4e6287e69478dd51d783,6


In [13]:
data1 = pd.merge(stock, data[['Gender', 'IdIssuePlace', 'Age', 'main_account']], how='left', left_on='main_account', right_on='main_account')
data1

,main_account,ShareCode,ShareBalance,Gender,IdIssuePlace,Age
0,0001898ab3584ff6a99ec11ffcb1f09c,CTG,1.0,F,CA Hà Nội,56
1,0001898ab3584ff6a99ec11ffcb1f09c,FLC,401.0,F,CA Hà Nội,56
2,0001898ab3584ff6a99ec11ffcb1f09c,PGC,5.0,F,CA Hà Nội,56
3,0001898ab3584ff6a99ec11ffcb1f09c,PVI,50.0,F,CA Hà Nội,56
4,0001898ab3584ff6a99ec11ffcb1f09c,PVS,2140.0,F,CA Hà Nội,56
...,...,...,...,...,...,...
42655,fff283aac89f4f82ab8b931dd9a9e41e,HUT,700.0,F,Cục CS ĐKQL CT và DLQG về DC,29
42656,fff283aac89f4f82ab8b931dd9a9e41e,PVV,1700.0,F,Cục CS ĐKQL CT và DLQG về DC,29
42657,fff283aac89f4f82ab8b931dd9a9e41e,VRG,12400.0,F,Cục CS ĐKQL CT và DLQG về DC,29
42658,fff942970c8d4976993aa8e949f2c228,IDI,7.0,M,CA TPHCM,29
